Импортируем необходимые библиотеки

In [10]:
import csv
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift

Преобразуем исходный файл в __checkins.csv__, убрав пустые строки и разделители

In [3]:
with open('checkins.dat', 'r') as fin:      
    newLines = []
    for line in fin:
        newLine = [x.strip() for x in line.split('|')]
        if len(newLine) == 6 and newLine[3] and newLine[4]:
            newLines.append(newLine)
            
with open('checkins.csv', 'w') as fout:
    file_writer = csv.writer(fout)
    file_writer.writerows(newLines)

Построим DataFrame и убедимся в целостности файла

In [54]:
dataset = pd.read_csv('checkins.csv')

In [55]:
dataset

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18
...,...,...,...,...,...,...
396629,955561,626076,20073,40.850100,-73.866246,2012-04-13 09:56:48
396630,955892,674797,2297,33.748995,-84.387982,2012-04-13 10:56:03
396631,956377,845102,11195,42.765366,-71.467566,2012-04-13 12:08:45
396632,956119,1139114,29488,42.439479,-83.743830,2012-04-13 11:36:44


В данном файле слишком много строк, поэтому будем использовать лишь первые 100 000

In [57]:
subset = dataset[['latitude', 'longitude']][:100000]

In [58]:
model = MeanShift(bandwidth = 0.1)

In [60]:
clustering = model.fit(subset)

Находим кластеры и количества элементов в них

In [106]:
labels_unique, counts_unique = np.unique(clustering.labels_, return_counts = True)

print('Количество кластеров: %d' % len(labels_unique))
print('Первые 10 меток кластеров:', labels_unique[:10])
print('Количество элементов в каждом из них:', counts_unique[:10])

Количество кластеров: 3231
Первые 10 меток кластеров: [0 1 2 3 4 5 6 7 8 9]
Количество элементов в каждом из них: [12506  4692  3994  3363  3526  2409  2297  1601  1526  1378]


Находим оптимальные кластеры - те, в которых более 15 элементов

In [107]:
labels_opt = [label for label in labels_unique if counts_unique[label] > 15]
print('Количество оптимальных кластеров:', len(labels_opt))

Количество оптимальных кластеров: 592


Записываем координаты офисов компании

In [83]:
offices = [[33.751277, -118.188740],
           [25.867736, -80.324116],
           [51.503016, -0.075479],
           [52.378894, 4.885084],
           [39.366487, 117.036146],
           [-33.868457, 151.205134]]

Для определения лучших мест размещения рекламы найдем расстояния от офисов компаний до центров кластеров

In [90]:
dists, labels = [], []
for i in range(len(offices)):
    for label in labels_opt:
        dists.append(np.linalg.norm(offices[i] - clustering.cluster_centers_[label]))
        labels.append(label)
        
print(dists[:5])
print(labels[:5])

[44.74257091784804, 6.193959165601315, 6.294241462747163, 31.621096640593514, 5.77204797983301]
[0, 1, 2, 3, 4]


Выберем 20 центров кластеров, наиболее близких к офисам компаний

In [116]:
dists_np = np.asarray(dists)

best_20_coords = []

for i in dists_np.argsort()[:20]:
    best_20_coords.append(clustering.cluster_centers_[labels[i]])

best_20_coords = np.asarray(best_20_coords)
print('20 ближайших лучших мест для размещения баннеров:')
print(best_20_coords)

best_coords = clustering.cluster_centers_[labels[np.argmin(dists_np)]]

print('\nБлижайшее лучшее место для размещения баннера:')
print(best_coords)

20 ближайших лучших мест для размещения баннеров:
[[-3.38606304e+01  1.51204776e+02]
 [ 5.23729640e+01  4.89231722e+00]
 [ 2.58456723e+01 -8.03188906e+01]
 [ 5.15029913e+01 -1.25537289e-01]
 [ 3.38098780e+01 -1.18148924e+02]
 [ 2.57858124e+01 -8.02179380e+01]
 [ 2.57053497e+01 -8.02834287e+01]
 [ 2.60100982e+01 -8.01999906e+01]
 [ 3.38883253e+01 -1.18048928e+02]
 [ 3.38729860e+01 -1.18362091e+02]
 [ 3.39725748e+01 -1.18168371e+02]
 [ 2.61388438e+01 -8.03343468e+01]
 [ 3.39839359e+01 -1.18007405e+02]
 [ 2.61208627e+01 -8.01589067e+01]
 [ 3.38173064e+01 -1.17891249e+02]
 [ 3.40603976e+01 -1.18248709e+02]
 [ 3.36743027e+01 -1.17858789e+02]
 [ 2.62005846e+01 -8.02507161e+01]
 [ 3.40354870e+01 -1.18438998e+02]
 [ 3.41314601e+01 -1.18118012e+02]]

Ближайшее лучшее место для размещения баннера:
[-33.86063043 151.20477593]


In [102]:
with open("banners_answer.txt", "w") as fout:
    fout.write(str(best_coords[0]) + ' ' + str(best_coords[1]))

In [118]:
with open("best_20_coords.txt", "w") as fout:
    for coord in best_20_coords:
        fout.write(str(coord[0]) + ' ' + str(coord[1]) + '\n')